# Notebook to compare Penn CHIME fits against all parameter fits

## Imports

In [1]:
from os import environ
from datetime import date

from pandas import DataFrame
from numpy import empty

from penn_chime.parameters import Parameters, Disposition
from penn_chime.models import (
    SimSirModel,
    sim_sir,
    calculate_admits,
    calculate_dispositions,
)
import penn_chime

from models import sir_step, sihr_step, one_minus_logistic_fcn, FitFcn

In [2]:
!pip show penn_chime

Name: penn-chime
Version: 1.1.3
Summary: COVID-19 Hospital Impact Model for Epidemics
Home-page: https://github.com/CodeForPhilly/chime
Author: Predictive Healthcare @ Penn Medicine
Author-email: 
License: UNKNOWN
Location: /Users/christopherkorber/gdrive/git/bochum/chime/src
Requires: altair, black, gunicorn, dash, dash-bootstrap-components, numpy, pandas, pytest, pyyaml, selenium, streamlit
Required-by: 


## Set up Penn CHIME model

In [3]:
p = Parameters(
    current_hospitalized=69,
    date_first_hospitalized=date(2020, 3, 7),
    doubling_time=4.0,
    hospitalized=Disposition.create(days=7, rate=0.025),
    icu=Disposition.create(days=9, rate=0.0075),
    infectious_days=14,
    market_share=0.15,
    n_days=100,
    population=3600000,
    recovered=0,
    relative_contact_rate=0.3,
    ventilated=Disposition.create(days=10, rate=0.005),
)

p.doubling_time = None
simsir = SimSirModel(p)

2020-04-08 19:25:02,177 - penn_chime.models - INFO - Using date_first_hospitalized: 2020-03-07; current_date: 2020-04-08; i_day: 32, current_hospitalized: 69
2020-04-08 19:25:02,258 - penn_chime.models - INFO - Estimated doubling_time: 5.116201582673885
2020-04-08 19:25:02,263 - penn_chime.models - INFO - len(np.arange(-i_day, n_days+1)): 133
2020-04-08 19:25:02,264 - penn_chime.models - INFO - len(raw_df): 133


## Tests

### Check that model agrees with Penn CHIME if no policies are in place

Calculate S, I, H, R for no policies

In [4]:
n_days = simsir.raw_df.day.max() - simsir.raw_df.day.min() + 1

policies = [(simsir.beta, n_days)]
raw = sim_sir(
    simsir.susceptible,
    simsir.infected,
    p.recovered,
    simsir.gamma,
    -simsir.i_day,
    policies,
)


calculate_dispositions(raw, simsir.rates, market_share=p.market_share)
calculate_admits(raw, simsir.rates)

raw_df = DataFrame(raw)

day0 = raw_df.iloc[0].fillna(0)

raw_df.head()

,day,susceptible,infected,recovered,ever_infected,ever_hospitalized,hospitalized,ever_icu,icu,ever_ventilated,ventilated,admits_hospitalized,admits_icu,admits_ventilated
0,-32,3.569996e+06,20155.529217,0.000000,20155.529217,75.583235,NaN,22.674970,NaN,15.116647,NaN,NaN,NaN,NaN
1,-31,3.565668e+06,23043.788678,1439.680658,24483.469336,91.813010,16.229775,27.543903,4.868933,18.362602,3.245955,16.229775,4.868933,3.245955
2,-30,3.560726e+06,26339.933063,3085.665564,29425.598627,110.345995,18.532985,33.103798,5.559895,22.069199,3.706597,18.532985,5.559895,3.706597
3,-29,3.555085e+06,30099.722589,4967.089354,35066.811943,131.500545,21.154550,39.450163,6.346365,26.300109,4.230910,21.154550,6.346365,4.230910
4,-28,3.548649e+06,34385.975372,7117.069539,41503.044911,155.636418,24.135874,46.690926,7.240762,31.127284,4.827175,24.135874,7.240762,4.827175


Compute values using new fit function

In [5]:
pars = {
    "beta_i": simsir.beta,
    "gamma_i": simsir.gamma,
    "initial_susceptible": day0.susceptible,
    "initial_infected": day0.infected,
    "initial_hospitalized": day0.hospitalized,
    "initial_recovered": day0.recovered,
    "hospitalization_rate": simsir.rates["hospitalized"] * p.market_share,
}
x = {
    "n_iter": raw_df.shape[0],
}


f = FitFcn(sir_step, columns=["susceptible", "infected", "hospitalized", "recovered"])
y = f(x, pars)

Check that difference is consistent with zero

In [6]:
mean = (y - raw_df[f.columns]).mean()
sdev = (y - raw_df[f.columns]).std()
assert (mean.abs() < 2 * sdev).all()
DataFrame([mean, sdev], index=["mean", "sdev"]).T

,mean,sdev
susceptible,2.454278e-11,3.252019e-10
infected,2.021250e-11,1.535918e-10
hospitalized,-6.203851e-14,1.550370e-12
recovered,-6.242052e-10,8.747967e-10


### Check that model agrees with Penn CHIME if no policies are in place

Now compare against Penn CHIME with active social distancing policies.

My fit function takes a function which returns an array of betas, this is implemented below

In [7]:
POLICIES = simsir.gen_policy(p)


def beta_i_fcn(n_iter, **kwargs):
    out = empty(n_iter)
    ii = 0
    for beta, n_days in POLICIES:
        for _ in range(n_days):
            out[ii] = beta
            ii += 1

    return out

In [8]:
day0 = simsir.raw_df.iloc[0]

pars = {
    "beta_i": simsir.beta,
    "gamma_i": simsir.gamma,
    "initial_susceptible": day0.susceptible,
    "initial_infected": day0.infected,
    "initial_hospitalized": day0.hospitalized,
    "initial_recovered": day0.recovered,
    "hospitalization_rate": simsir.rates["hospitalized"] * p.market_share,
}
x = {
    "n_iter": simsir.raw_df.shape[0],
}


f = FitFcn(
    sir_step,
    beta_i_fcn=beta_i_fcn,
    columns=["susceptible", "infected", "hospitalized", "recovered"],
)
y = f(x, pars)

In [9]:
mean = (y - simsir.raw_df[f.columns]).mean()
sdev = (y - simsir.raw_df[f.columns]).std()
assert (mean.abs() < 2 * sdev).all()
DataFrame([mean, sdev], index=["mean", "sdev"]).T

,mean,sdev
susceptible,-1.076623e-09,4.243765e-09
infected,1.132751e-09,1.605516e-09
hospitalized,2.812527e-13,1.172452e-12
recovered,4.475682e-09,4.921139e-09
